In [1]:
import os

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere, CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

# to handle conversational memory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from oci_cohere_embeddings_utils import OCIGenAIEmbeddingsWithBatch
from utils import format_docs
from my_prompts import prompt_4_answer

from config import EMBED_MODEL, ENDPOINT, COHERE_GENAI_MODEL
from config_private import COMPARTMENT_OCID, COHERE_API_KEY, LANGSMITH_API_KEY

In [2]:
#
# abilitiamo il tracing con LangSmith
#
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "memory01"
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY

# parametri per il retrieval
# Similarity search
TOP_K = 6
# reranking
TOP_N = 4

# la directory che contiene i pdf
DIR_NAME = "./books"
# la directory in cui il vector store è salvato
FAISS_DIR = "./faiss_index"

# parametri LLM Cohere
TEMPERATURE = 0.0
MAX_TOKENS = 1024

In [3]:
embed_model = OCIGenAIEmbeddingsWithBatch(
    auth_type="API_KEY",
    model_id=EMBED_MODEL,
    service_endpoint=ENDPOINT,
    compartment_id=COMPARTMENT_OCID,
)

cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY, top_n=TOP_N)

#
# LLM
#
llm = ChatCohere(
    cohere_api_key=COHERE_API_KEY,
    model=COHERE_GENAI_MODEL,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
)

In [4]:
#
# qui vettorizza tutti i chunks oppure
# li legge se salvati in locale
#
if os.path.exists(FAISS_DIR):
    print("Loading Vector Store from local dir...")

    v_store = FAISS.load_local(
        FAISS_DIR, embed_model, allow_dangerous_deserialization=True
    )
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False,
    )

    loader = PyPDFDirectoryLoader(path=DIR_NAME, glob="*.pdf")

    # document are pages
    docs = loader.load_and_split(text_splitter=text_splitter)

    print(f"Loaded {len(docs)} chunks...")

    print(f"Indexing books...")
    v_store = FAISS.from_documents(docs, embed_model)

    v_store.save_local(FAISS_DIR)

Loaded 2809 chunks...
Indexing books...


  0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
base_retriever = v_store.as_retriever()

#
# add a reranker
#
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, base_retriever=base_retriever
)

In [7]:
#
# create the entire chain
#
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, output_key="answer"
)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=compression_retriever,
    memory=memory,
    # cosi customizzo il final prompt
    combine_docs_chain_kwargs={"prompt": prompt_4_answer},
    return_source_documents=True,
)

In [16]:
def answer(chain, question):
    response = chain.invoke(question)

    print(question)
    print("")
    print(response["answer"])

In [24]:
%%time
question = """Scrivi una mail, da inviare ad un cliente di nome Luigi Saetta, 
           in cui spieghi quali vantaggi Oracle Data Guard offre per il Disaster Recovery"""

answer(rag_chain, question)

  0%|          | 0/1 [00:00<?, ?it/s]

Scrivi una mail, da inviare ad un cliente di nome Luigi Saetta, 
           in cui spieghi quali vantaggi Oracle Data Guard offre per il Disaster Recovery

I vantaggi chiave di Oracle Data Guard in termini di protezione dei dati e di disaster recovery sono i seguenti:
- Replicazione dei dati in tempo reale: Oracle Data Guard crea e mantiene una o più repliche in standby della tua database principale. Queste repliche in standby sono aggiornate in tempo reale, riducendo al minimo la perdita di dati in caso di interruzione.
- Flessibilità: Supporta vari tipi di database standby, tra cui fisico, snapshot e logico, consentendoti di scegliere l'opzione più adatta alle tue esigenze. La flessibilità consente inoltre di utilizzare i database standby per carichi di lavoro read-only, riducendo il carico sul database primario.
- Scalabilità: Fino a 30 database standby possono ricevere il redo direttamente dal database primario. Inoltre, la possibilità di utilizzare un database standby a cascata co

In [25]:
question = """Qual farmaci possono essere usati per curare il diabete di tipo 2 negli adulti?"""

answer(rag_chain, question)

  0%|          | 0/1 [00:00<?, ?it/s]

Qual farmaci possono essere usati per curare il diabete di tipo 2 negli adulti?

I farmaci disponibili per il trattamento del diabete di tipo 2 negli adulti sono numerosi. Il primo farmaco di scelta per il trattamento del diabete di tipo 2 è in genere la metformina, che appartiene alla classe dei biguanidi. La metformina aiuta il corpo a ripristinare la sua capacità di rispondere all'insulina e abbassa i livelli di zucchero nel sangue inibendo la produzione di glucosio nel fegato. 

Altri farmaci che possono essere prescritti sono:

- Inibitori delle alfa-glucosidasi, come l'acarbose e la miglitol, che rallentano l'assorbimento degli zuccheri nell'intestino tenue
- Sulfoniluree, come la glimepiride e la glipizide, che stimolano le cellule pancreatiche a rilasciare più insulina
- Glitazoni, come la pioglitazone, che aumentano la sensibilità dell'organismo all'insulina
- Inibitori del co-trasportatore 2 del sodio-glucosio, come la dapagliflozina e l'empagliflozina, che riducono la riasso

In [20]:
question = """Può essere opportuno usare la metformina nei pazienti anziani?"""

answer(rag_chain, question)

  0%|          | 0/1 [00:00<?, ?it/s]

Può essere opportuno usare la metformina nei pazienti anziani?

Si', la metformina e' generalmente considerata una scelta sicura per il trattamento del diabete di tipo 2 nei pazienti anziani. E' un farmaco raccomandato nelle linee guida sia americane che europee per il trattamento del diabete di tipo 2 in pazienti anziani, a causa del suo profilo di sicurezza favorevole e del suo basso costo. 

Tuttavia, e' importante utilizzare la metformina con cautela in alcuni pazienti anziani con problemi di salute aggiuntivi. La metformina deve essere usata con cautela nei pazienti anziani con insufficienza renale, poiché può accumularsi nel corpo e causare effetti collaterali. I pazienti anziani possono anche essere più sensibili agli effetti collaterali della metformina, come nausea, vomito e diarrea. Quindi, la dose deve essere attentamente monitorata e aggiustata in base alle condizioni individuali.

Inoltre, la metformina deve essere interrotta prima di qualsiasi intervento chirurgico o proc

In [ ]:
memory.clear()